<a href="https://colab.research.google.com/github/JosepCristobal/NLPSearchPT/blob/master/NLPSearchPTv01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NLP_Search_PTv01

### 1. Machine Learning vs Deep Learning (Acordaros que hay que implementar el pipeline visto en clase entero)

    1.1. Implementación de un modelo de Sentiment Analysis con algún algoritmo de Machine Learning Clásico.

    1.2. Implementación de un modelo de Sentiment Analysis con alguna arquitectura de Deep Learning.

    1.3. Breve Comparación de resultados. Confusion Matrix.

Procedemos a los imports y las instalaciones que inicialmente creemos necesarias

In [0]:
!pip install spacy

In [102]:
!pip install stop_words

In [2]:
!ls

sample_data


In [103]:
!python -m spacy download es_core_news_md

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [4]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 95.4MB 65.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=18f50208b93370b30d3874bc1593cb9ccc2cb38776e6f4256d94256c2be7863e
  Stored in directory: /tmp/pip-ephem-wheel-cache-cty575ob/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [105]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [104]:
!python -m spacy download es_core_news_md

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [0]:
import spacy
from spacy import displacy

In [0]:
nlp = spacy.load('en_core_web_sm')

Empezamos a importar datos para el sentiment analisys

In [0]:
%matplotlib inline

import numpy as np

import pickle
import json
import os
import csv

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from random import sample

import warnings
warnings.filterwarnings('ignore')

import io

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [0]:
def split_train_val_test(dataset, split=0.2):

    x, y = zip(*dataset)
    x = np.array(list(x))
    y = np.array(list(y))
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=1337) #l33t seed
    for train_index, test_index in sss.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
    splits = {'train':(x_train, y_train), 'test':(x_val, y_val)}
    return splits

In [0]:
from google.colab import files

In [113]:
#Copiamos en nuestro directorio local los ficheros de datos para la práctica. Renombramos train_sentiment (1).csv a train_sentiment.csv
#Cargamos nuestro dataset desde nuestro diretorio local a través de la selección de fichero en cuadro de dialogo.
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train_sentiment.csv to train_sentiment (4).csv
User uploaded file "train_sentiment.csv" with length 8664015 bytes


In [114]:
#En este caso, cargaremos un fichero en formato csv y con página de códigos 'ISO-8859-1' a nuestro dataset
df = pd.read_csv(io.StringIO(uploaded['train_sentiment.csv'].decode('ISO-8859-1')))

df.head(5)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [115]:
#Procedemos a la limpieza de los datos. Primeramente borramos espacios en blanco delante y detras de cada registro de 'SentimentText'
df['SentimentText'] = df['SentimentText'].str.strip()
df.head(3)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL friend.............
1,2,0,I missed the New Moon trailer...
2,3,1,omg its already 7:30 :O


In [116]:
#MAS LIMPIEZA
#Procedemos a borrar registros nulos
df.dropna(subset=['SentimentText'],inplace=True)
df.head(3)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL friend.............
1,2,0,I missed the New Moon trailer...
2,3,1,omg its already 7:30 :O


In [0]:
#Montaremos una serie de filtros para terminar de limpiar nuestros datos
import re
import string
#Utilizamos regex para limpiar nuestro dataset de carácteres especiales, signos de puntuación , etc. 
def clean_text_round1(text):
    #Transformamos las palabras en minusculas, eliminamos los corchete, eliminamos signos de puntuación y palabras con números.
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    #Eliminamos signos de puntuación adicionales que en la primera vuelta no fueron tratados
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)

    return text
 


In [118]:
df.columns[0:]

Index(['ItemID', 'Sentiment', 'SentimentText'], dtype='object')

In [119]:
df.isnull().any()

ItemID           False
Sentiment        False
SentimentText    False
dtype: bool

In [120]:
#Limpiamos datos y transformamos en texto 1,0 POSITIVE, NEGATIVE respectivamente
#Para la limpieza, invocamos la función creada para ello.
#También aplicamos un filtro utilizando stop words
from stop_words import get_stop_words
pkg_stop_words = get_stop_words('en')
new_dataset = []
for index, row in df.iterrows():
    if index>0:
        label = 'POSITIVE' if row[1] == 1 else 'NEGATIVE' 
        sentence = clean_text_round1(row[2])
        sentence_result=' '.join([word for word in sentence.split() if word not in pkg_stop_words])
        new_dataset.append((sentence_result, label))
print(new_dataset[0])
len(new_dataset)

('missed new moon trailer', 'NEGATIVE')


99988

In [0]:
#Limpieza para la red neuronal
#Para la limpieza, invocamos la función creada para ello.
#También aplicamos un filtro utilizando stop words
from stop_words import get_stop_words
pkg_stop_words = get_stop_words('en')

new_datasetNN = []
for index, row in df.iterrows():
    if index>0:
        label = 1 if row[1] == 1 else 0
        sentence = clean_text_round1(row[2])
        sentence_result=' '.join([word for word in sentence.split() if word not in pkg_stop_words])
        new_datasetNN.append((sentence_result, label))


In [0]:
#Para pruebas SVG seleccionamos 5000 registros para no eternizar las pruebas
#new_dataset=new_dataset[:1000]
new_datasetNN=new_datasetNN[:5000]

In [0]:
split = split_train_val_test(new_dataset, 0.20)


In [130]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    #('clf', SVC(class_weight='balanced', probability=True)) 
    #MultinomialNB Bayes, Para usarla, comentariamos la linea anterior y descomentariamos la siguiente
    ('clf', MultinomialNB()), #MultinomialNB
])
pipeline.get_params().keys() # que parametros podemos tocar en el gridsearch!

dict_keys(['memory', 'steps', 'verbose', 'vect', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'clf__alpha', 'clf__class_prior', 'clf__fit_prior'])

In [0]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 2), (1, 3)), #(2, 3), (1,4)),  #ngrams para el test
    'vect__analyzer': ('word', 'char') #Buscamos los más óptimo, analisis por palabras o por carácteres.
}

In [0]:
#Configuramos nuestro GridSearch para buscar los mejores parámetros.
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [133]:
#Lo ejecutamos
grid_search.fit(split['train'][0], split['train'][1])
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  4.1min finished


	vect__analyzer: 'word'
	vect__max_df: 0.5
	vect__ngram_range: (1, 2)
Best score: 0.748


In [134]:
#Asignamos los mejores parámetros a nuestro Pipeline
pipeline.set_params(**best_parameters)
predictions = pipeline.score(split['test'][0], split['test'][1])
print('TEST SCORE: {}'.format(predictions))

#El resultado inicial sin hacer demasiada limpieza y con pocos filtos ha sido TEST SCORE: 0.68, a ver si lo mejoramos
#En la última prueba, con todos los filtros aplicados, hemos conseguido un TEST SCORE: 0.7566256625662566

TEST SCORE: 0.7566256625662566


In [0]:
#new_dataset


In [0]:
is_positive = sample([x for x, y in new_dataset if y=='POSITIVE'], 5)

In [136]:
p_positive = sample(list(split['test'][0]), 10)
p_positive

['alimsalahuddin lol m going see maybe today',
 'acidargyle m making list people rock yeah pretty much make entire list awesomeupdater',
 'afzalalmighty friends sad',
 'goodsex topmost trend today',
 'squarespace birthday saturday shame driving',
 'supernatural now now don t call twitter stupid everyone overloading gonna blow one days nice',
 'cousins leaving thursday',
 'albumoftheweek name sunset rubdown s newest album dragonslayer album week cause frickin love',
 'supernatural http twitpic com oh love',
 'lt poor spent expected makeup show lmaoo much project pan hahaha']

In [137]:
predictions = pipeline.predict(is_positive)
predictions_probs = pipeline.predict_proba(is_positive)
for x, y, p in zip(p_positive, predictions, predictions_probs):
    print('{} ------> {} {}'.format(x, y, p)) # p

alimsalahuddin lol m going see maybe today ------> POSITIVE [5.52492403e-05 9.99944751e-01]
acidargyle m making list people rock yeah pretty much make entire list awesomeupdater ------> POSITIVE [0.00577458 0.99422542]
afzalalmighty friends sad ------> POSITIVE [8.36685470e-04 9.99163315e-01]
goodsex topmost trend today ------> POSITIVE [8.26280371e-05 9.99917372e-01]
squarespace birthday saturday shame driving ------> POSITIVE [1.12990772e-06 9.99998870e-01]


###1.2. Implementación de un modelo de Sentiment Analysis con alguna arquitectura de Deep Learning.

In [138]:
new_datasetNN[:10]

[('missed new moon trailer', 0),
 ('omg already o', 1),
 ('omgaga im sooo im gunna cry ve dentist since suposed just get crown put',
  0),
 ('think mi bf cheating t t', 0),
 ('just worry much', 0),
 ('juuuuuuuuuuuuuuuuussssst chillin', 1),
 ('sunny work tomorrow tv tonight', 0),
 ('handed uniform today miss already', 1),
 ('hmmmm wonder number', 1),
 ('must think positive', 0)]

In [0]:
labels = ['NEGATIVE',
            'POSITIVE'
         ]

In [0]:
from keras.layers import Conv1D, Conv2D, SimpleRNN, LSTM, Dense, Dropout, CuDNNLSTM
from keras.models import Sequential, Model
from collections import Counter
from keras import backend as K
from keras.layers import Input, Average, average, Lambda
from keras.layers import Embedding
from keras.layers import Flatten, Input
from keras.models import Model


In [141]:
tokens = list() # lista con todos los tokens que hay en el dataset
tokenized = list() # lista con tuplas (documento_tokenizado, etiqueta)

for x, y in new_datasetNN:
    toks = [t.text for t in nlp(x)]
    tokens += toks
    tokenized.append((toks, y))

vocab_counter = Counter(tokens)
vocab = list(vocab_counter.keys())
len(vocab), len(tokenized)

(8771, 5000)

In [142]:
maxlen = max([len(x) for x, _ in tokenized])
maxlen

32

In [143]:
lens = [len(x) for x, _ in tokenized]
median = np.median(np.array(lens))
mean = np.mean(np.array(lens))
maxlen = int(median)*2
print(median, mean, maxlen)
#Doble de la mediana para la dimensión máxima

7.0 7.4694 14


In [144]:
tokenized_filtered = [(x, y) for x, y in tokenized if len(x)<maxlen]
len(tokenized_filtered)

4347

In [0]:
w2id = {k:i for i, k in enumerate(vocab)}
w2id['<UNK>'] = len(w2id)



In [0]:
l2id = {label:i for i, label in enumerate(labels)}

In [147]:
MIN_APPEARENCE_TOKEN = 5
#Palabras que han aparecido menos de X veces
input_ready = list()
for x, y in tokenized_filtered:
    sentence = np.zeros((maxlen))
    label = np.zeros((len(labels)))
    #label[int(y) - 1] = 1 En nuestro caso valor mínimo es 0 no uno como en el ejemplo de clase
    label[int(y)] = 1
    #Transformamos de tokens a sus ids
    for i, t in enumerate(x):
        sentence[i] = w2id[t] if t in vocab_counter and vocab_counter[t]>= MIN_APPEARENCE_TOKEN else w2id['<UNK>']

    input_ready.append((sentence, label))

input_ready[0][0].shape

(14,)

In [148]:
splits = split_train_val_test(input_ready)
splits['train'][0].shape, splits['train'][1].shape, splits['test'][0].shape,splits['test'][1].shape

((3477, 14), (3477, 2), (870, 14), (870, 2))

In [0]:
#Implementamos DAN Deep Averaging Networks

In [0]:
def mask_aware_mean(x):
    #Macamos las filas que todo son ceros
    mask = K.not_equal(K.sum(K.abs(x), axis=2, keepdims=True), 0)
    #Calculamos los que nos interesa quedarnos
    n = K.sum(K.cast(mask, 'float32'), axis=1, keepdims=False)
    x_mean = K.sum(x, axis=1, keepdims=False) / n
    return x_mean

def mask_aware_mean_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3, 'Dimensions for average are wrong'
    return (shape[0], shape[2])
    

In [150]:
input_layer = Input(shape=(maxlen,))
embedding = Embedding(output_dim=100, input_dim=len(w2id), input_length=maxlen, name='m_embedding')(input_layer)
doc_representation = Lambda(mask_aware_mean, mask_aware_mean_output_shape, name='embedding_average')(embedding)
dense_1 = Dense(100, activation='relu')(doc_representation)
drop_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(100, activation='relu')(drop_1)
drop_2 = Dropout(0.5)(dense_2)
out = Dense(len(l2id), activation='softmax')(drop_2)

model = Model(inputs=input_layer, outputs=out)
model.compile(
    loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 14)                0         
_________________________________________________________________
m_embedding (Embedding)      (None, 14, 100)           877200    
_________________________________________________________________
embedding_average (Lambda)   (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0   

In [151]:
model.fit(splits['train'][0], splits['train'][1], epochs=115, batch_size=16)

Epoch 1/115
3477/3477 [==============================] - 3s 756us/step - loss: 0.5991 - acc: 0.7124
Epoch 2/115
3477/3477 [==============================] - 2s 673us/step - loss: 0.5319 - acc: 0.7225
Epoch 3/115
3477/3477 [==============================] - 2s 698us/step - loss: 0.4765 - acc: 0.7820
Epoch 4/115
3477/3477 [==============================] - 2s 706us/step - loss: 0.4434 - acc: 0.7967
Epoch 5/115
3477/3477 [==============================] - 2s 690us/step - loss: 0.4136 - acc: 0.8223
Epoch 6/115
3477/3477 [==============================] - 2s 706us/step - loss: 0.4053 - acc: 0.8131
Epoch 7/115
3477/3477 [==============================] - 3s 729us/step - loss: 0.3876 - acc: 0.8263
Epoch 8/115
3477/3477 [==============================] - 2s 678us/step - loss: 0.3792 - acc: 0.8349
Epoch 9/115
3477/3477 [==============================] - 2s 657us/step - loss: 0.3718 - acc: 0.8335
Epoch 10/115
3477/3477 [==============================] - 2s 697us/step - loss: 0.3641 - acc: 0.8401

In [152]:
model.evaluate(splits['test'][0], splits['test'][1])

870/870 [==============================] - 0s 128us/step


[1.890613648997775, 0.7494252873563219]

Confusion Matrix

In [0]:
y_pred = model.predict(splits['test'][0])
y_pred = (y_pred > 0.5) #greater than 0.50 on scale 0 to 1
print(y_pred)

#Making confusion matrix that checks accuracy of the model
#rounded_labels=np.argmax(test_labels, axis=1)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(splits['test'][1], y_pred)

In [0]:
#No soy capaz de ponerlo en marcha, estoy en un bucle. Lo dejo para más adelante a ver si veo como conseguirlo.